# CLASSIFICATION - SVM (Support Vector Machine)
## Classification des prix immobiliers (House-Data.csv)
### OMGBA Joseph

**Objectif**: Classifier les maisons en 3 catégories de prix (Bas/Moyen/Élevé)

**Justification du choix de la classification**:
> Dans la pratique immobilière, les acheteurs cherchent souvent une **gamme de prix** plutôt qu'un prix exact. Classifier en catégories permet de :
> - Simplifier la décision d'achat
> - Évaluer les performances avec des métriques claires (accuracy, ROC)
> - Utiliser le modèle SVM dans sa forme classique (SVC)

# PREPROCESSING

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import (confusion_matrix, classification_report, accuracy_score,
                             RocCurveDisplay, roc_auc_score, f1_score)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Charger les données
data = pd.read_csv('House-Data.csv')

print("--- Aperçu des données ---")
print(data.head())
print("\nDimensions :", data.shape)
print("\nTypes de données :\n", data.dtypes)

In [ ]:
# Vérifier les valeurs manquantes
print("\nValeurs manquantes par colonne :")
print(data.isna().sum())

# Remplacer par la moyenne si nécessaire
for col in data.select_dtypes(include=[np.number]).columns:
    if data[col].isnull().any():
        data[col].fillna(data[col].mean(), inplace=True)
        print(f"Valeurs manquantes dans {col} remplacées par la moyenne.")

In [ ]:
# Vérification et suppression des doublons
print("\nNombre de doublons :", data.duplicated().sum())
data.drop_duplicates(inplace=True)
print(f"Dimensions après suppression : {data.shape}")

In [ ]:
# Supprimer les colonnes non pertinentes pour la prédiction
columns_to_drop = ['id', 'date']
data = data.drop(columns=columns_to_drop, errors='ignore')
print(f"Colonnes supprimées : {columns_to_drop}")
print(f"Nouvelles dimensions : {data.shape}")

In [ ]:
# Description statistique
print("\nDescription statistique :")
data.describe()

In [ ]:
# Distribution des prix
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(data['price'], bins=50, color='steelblue', edgecolor='black')
plt.title('Distribution des prix')
plt.xlabel('Prix ($)')
plt.ylabel('Fréquence')

plt.subplot(1, 2, 2)
plt.boxplot(data['price'])
plt.title('Boxplot des prix')
plt.ylabel('Prix ($)')

plt.tight_layout()
plt.show()

In [ ]:
# Matrice de corrélation
plt.figure(figsize=(14, 10))
correlation_matrix = data.select_dtypes(include=['number']).corr()
sns.heatmap(correlation_matrix, annot=True, cmap='RdYlGn', fmt='.2f', annot_kws={'size': 8})
plt.title("Matrice de Corrélation")
plt.tight_layout()
plt.show()

## Création de la variable cible (Catégories de prix)

In [ ]:
# Définir les catégories de prix
def categorize_price(price):
    if price < 300000:
        return 0  # Prix Bas
    elif price < 600000:
        return 1  # Prix Moyen
    else:
        return 2  # Prix Élevé

# Appliquer la catégorisation
data['price_category'] = data['price'].apply(categorize_price)

# Afficher la distribution des catégories
print("Distribution des catégories de prix :")
print(data['price_category'].value_counts().sort_index())

# Labels pour l'affichage
category_labels = {0: 'Prix Bas (<300k)', 1: 'Prix Moyen (300k-600k)', 2: 'Prix Élevé (>600k)'}
for cat, label in category_labels.items():
    count = (data['price_category'] == cat).sum()
    print(f"  {cat} - {label}: {count} ({count/len(data)*100:.1f}%)")

In [ ]:
# Visualiser la distribution des catégories
plt.figure(figsize=(8, 6))
colors = ['#3498db', '#2ecc71', '#e74c3c']
data['price_category'].value_counts().sort_index().plot(kind='bar', color=colors)
plt.title('Distribution des catégories de prix')
plt.xlabel('Catégorie')
plt.ylabel('Nombre de maisons')
plt.xticks([0, 1, 2], ['Bas (<300k)', 'Moyen (300k-600k)', 'Élevé (>600k)'], rotation=0)
plt.tight_layout()
plt.show()

## Séparation des données

In [ ]:
# Séparer les features et la cible
X = data.drop(['price', 'price_category'], axis=1)
y = data['price_category']

print(f"Shape de X: {X.shape}")
print(f"Shape de y: {y.shape}")
print(f"\nFeatures utilisées: {list(X.columns)}")

In [ ]:
# Division Train/Test (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"\nDistribution y_train: {y_train.value_counts().to_dict()}")
print(f"Distribution y_test: {y_test.value_counts().to_dict()}")

In [ ]:
# Standardisation (critique pour SVM)
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

print("Standardisation effectuée.")

# MODELE SVM CLASSIQUE

In [ ]:
# Création et entraînement du modèle SVM classique
classifier = SVC(kernel='linear', random_state=0, probability=True)
classifier.fit(X_train_sc, y_train)

print("Modèle SVM entraîné avec succès.")

### Critères d'évaluation du modèle classique

In [ ]:
# Prédictions
y_pred_train = classifier.predict(X_train_sc)
y_pred_test = classifier.predict(X_test_sc)

print("--- Performance du Modèle Classique ---")
print(f"Accuracy sur train: {accuracy_score(y_train, y_pred_train):.4f}")
print(f"Accuracy sur test: {accuracy_score(y_test, y_pred_test):.4f}")

In [ ]:
# 1 - Matrice de confusion
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Bas', 'Moyen', 'Élevé'],
            yticklabels=['Bas', 'Moyen', 'Élevé'])
plt.title('Matrice de confusion - SVM Classique')
plt.xlabel('Prédit')
plt.ylabel('Réel')
plt.tight_layout()
plt.show()

In [ ]:
# 2 - Classification report
print("Classification Report - SVM Classique:")
print(classification_report(y_test, y_pred_test, target_names=['Prix Bas', 'Prix Moyen', 'Prix Élevé']))

In [ ]:
# 3 - Courbe ROC (One vs Rest pour multiclasse)
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
class_names = ['Prix Bas', 'Prix Moyen', 'Prix Élevé']

for i, (ax, class_name) in enumerate(zip(axes, class_names)):
    RocCurveDisplay.from_estimator(
        classifier, X_test_sc, y_test,
        pos_label=i, ax=ax, name=f'Classe {i}'
    )
    ax.set_title(f'Courbe ROC - {class_name}')

plt.tight_layout()
plt.show()

In [ ]:
# 4 - Calcul de l'AUC-ROC global (One vs Rest)
y_pred_proba = classifier.predict_proba(X_test_sc)
auc_ovr = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='weighted')
print(f"AUC-ROC (One vs Rest, weighted): {auc_ovr:.4f}")

# OPTIMISATION DU MODELE SVM

In [ ]:
# Échantillonnage pour accélérer GridSearchCV (SVM est lent sur gros datasets)
sample_size = min(5000, len(X_train_sc))
np.random.seed(42)
sample_idx = np.random.choice(len(X_train_sc), sample_size, replace=False)
X_train_sample = X_train_sc[sample_idx]
y_train_sample = y_train.iloc[sample_idx]

print(f"Taille de l'échantillon pour GridSearchCV: {sample_size}")

In [ ]:
# Définir les hyperparamètres à optimiser
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}

print("Paramètres à tester:", param_grid)

In [ ]:
# GridSearchCV
grid_search = GridSearchCV(
    estimator=SVC(probability=True, random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_sample, y_train_sample)

In [ ]:
# Meilleurs paramètres
print("Meilleurs paramètres:")
print(grid_search.best_params_)
print(f"\nMeilleur score (CV): {grid_search.best_score_:.4f}")

In [ ]:
# Entraîner le modèle final avec les meilleurs paramètres sur TOUTES les données
best_svm = SVC(
    **grid_search.best_params_,
    probability=True,
    random_state=42
)
best_svm.fit(X_train_sc, y_train)

print("Modèle optimisé entraîné sur l'ensemble des données d'entraînement.")

### Critères d'évaluation du modèle optimisé

In [ ]:
# Prédictions avec le modèle optimisé
y_pred_train_opt = best_svm.predict(X_train_sc)
y_pred_test_opt = best_svm.predict(X_test_sc)

print("--- Performance du Modèle Optimisé ---")
print(f"Accuracy sur train: {accuracy_score(y_train, y_pred_train_opt):.4f}")
print(f"Accuracy sur test: {accuracy_score(y_test, y_pred_test_opt):.4f}")

In [ ]:
# 1 - Matrice de confusion - Modèle optimisé
cm_opt = confusion_matrix(y_test, y_pred_test_opt)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_opt, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Bas', 'Moyen', 'Élevé'],
            yticklabels=['Bas', 'Moyen', 'Élevé'])
plt.title('Matrice de confusion - SVM Optimisé')
plt.xlabel('Prédit')
plt.ylabel('Réel')
plt.tight_layout()
plt.show()

In [ ]:
# 2 - Classification report - Modèle optimisé
print("Classification Report - SVM Optimisé:")
print(classification_report(y_test, y_pred_test_opt, target_names=['Prix Bas', 'Prix Moyen', 'Prix Élevé']))

In [ ]:
# 3 - Courbe ROC - Modèle optimisé
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
class_names = ['Prix Bas', 'Prix Moyen', 'Prix Élevé']

for i, (ax, class_name) in enumerate(zip(axes, class_names)):
    RocCurveDisplay.from_estimator(
        best_svm, X_test_sc, y_test,
        pos_label=i, ax=ax, name=f'Classe {i}'
    )
    ax.set_title(f'Courbe ROC Optimisée - {class_name}')

plt.tight_layout()
plt.show()

In [ ]:
# 4 - Calcul de l'AUC-ROC global - Modèle optimisé
y_pred_proba_opt = best_svm.predict_proba(X_test_sc)
auc_ovr_opt = roc_auc_score(y_test, y_pred_proba_opt, multi_class='ovr', average='weighted')
print(f"AUC-ROC Optimisé (One vs Rest, weighted): {auc_ovr_opt:.4f}")

## Comparaison des modèles

In [ ]:
# Comparaison des performances
print("=" * 60)
print("COMPARAISON DES MODELES SVM")
print("=" * 60)
print(f"\nModèle Classique (kernel=linear):")
print(f"  - Accuracy train: {accuracy_score(y_train, y_pred_train):.4f}")
print(f"  - Accuracy test: {accuracy_score(y_test, y_pred_test):.4f}")
print(f"  - AUC-ROC: {auc_ovr:.4f}")
print(f"\nModèle Optimisé ({grid_search.best_params_}):")
print(f"  - Accuracy train: {accuracy_score(y_train, y_pred_train_opt):.4f}")
print(f"  - Accuracy test: {accuracy_score(y_test, y_pred_test_opt):.4f}")
print(f"  - AUC-ROC: {auc_ovr_opt:.4f}")
print(f"\nMeilleurs paramètres: {grid_search.best_params_}")

# PREDICTION AVEC LE MODELE OPTIMISE

In [ ]:
# Exemple de prédiction
sample = X_test.iloc[0:5]
sample_sc = sc.transform(sample)

predictions = best_svm.predict(sample_sc)
predictions_proba = best_svm.predict_proba(sample_sc)

category_names = ['Prix Bas (<300k)', 'Prix Moyen (300k-600k)', 'Prix Élevé (>600k)']

print("Prédictions pour 5 échantillons:")
for i, (pred, proba) in enumerate(zip(predictions, predictions_proba)):
    real = y_test.iloc[i]
    print(f"  {i+1}. Prédit: {category_names[pred]} | Réel: {category_names[real]} | Confiance: {max(proba)*100:.1f}%")

# SAUVEGARDE DU MODELE

In [ ]:
import pickle

# Sauvegarder le modèle, le scaler et les features
model_data = {
    'model': best_svm,
    'scaler': sc,
    'features': list(X.columns),
    'category_names': category_names,
    'best_params': grid_search.best_params_
}

with open('houseSVM.pkl', 'wb') as f:
    pickle.dump(model_data, f)

print("Modèle sauvegardé dans 'houseSVM.pkl'")
print(f"Features: {list(X.columns)}")

# RÉSUMÉ FINAL

In [ ]:
print("=" * 70)
print("                         RÉSUMÉ FINAL")
print("=" * 70)
print(f"""
DATASET:
  - Fichier: House-Data.csv
  - Dimensions: {data.shape}
  - Split: 70% train / 30% test

CLASSIFICATION:
  - Catégorie 0: Prix Bas (<300k$)
  - Catégorie 1: Prix Moyen (300k-600k$)
  - Catégorie 2: Prix Élevé (>600k$)

PREPROCESSING:
  - Suppression colonnes: id, date
  - StandardScaler appliqué

OPTIMISATION:
  - GridSearchCV avec échantillonnage ({sample_size} lignes)
  - Meilleurs paramètres: {grid_search.best_params_}

PERFORMANCES FINALES:
  - Accuracy:  {accuracy_score(y_test, y_pred_test_opt):.4f}
  - AUC-ROC:   {auc_ovr_opt:.4f}
  - F1-Score:  {f1_score(y_test, y_pred_test_opt, average='weighted'):.4f}

FICHIER SAUVEGARDÉ: houseSVM.pkl
""")
print("=" * 70)